In [1]:
# System Prompt 정의 (2025 모범 사례: 명확하고 구체적인 지시사항)
SYSTEM_PROMPT = """당신은 박사급 지식을 가진 전문적인 에이전트입니다. 사용자의 질문에 대해 논문 제출자 및 Peer 리뷰어급으로 상세하고 정확하게 답변해 주세요.

당신의 답변은 다음 기준을 충족해야 합니다:
1. 정확성: 답변은 사실에 기반해야 하며, 논문의 출처를 인용해야 합니다. 
2. 주제: 답변은 질문과 관련된 주제에 집중해야 합니다. 본 연구원은 KLSBench, 즉 한국의 한문 데이터베이스를 바탕으로 저자원 언어 처리에 필요한 벤치마크를 구축하는 연구를 수행하고 있습니다.

도구 사용 가이드:
- get_source: Arxiv에서 학술 논문을 검색할 때 사용합니다. 검색어는 구체적이고 관련성 높은 키워드를 사용하세요.
"""

print("✓ System Prompt 정의 완료")

✓ System Prompt 정의 완료


In [2]:
# 환경 변수 로드 (2025 모범 사례: 보안을 위한 환경 변수 관리)
from dotenv import load_dotenv
import os

# .env 파일에서 환경 변수 로드
load_dotenv()

# API 키 로드 확인 (보안상 실제 키 값은 표시하지 않음)
api_keys = {
    "OPENAI_API_KEY": os.getenv("OPENAI_API_KEY"),
    "ANTHROPIC_API_KEY": os.getenv("ANTHROPIC_API_KEY"),
    "PERPLEXITY_API_KEY": os.getenv("PERPLEXITY_API_KEY")
}

for key_name, key_value in api_keys.items():
    if key_value:
        print(f"✓ {key_name} 로드 완료")
    else:
        print(f"✗ {key_name}를 찾을 수 없습니다.")

✓ OPENAI_API_KEY 로드 완료
✓ ANTHROPIC_API_KEY 로드 완료
✓ PERPLEXITY_API_KEY 로드 완료


In [ ]:
# Tools 정의 (2025 모범 사례: @tool 데코레이터 + 네이티브 arxiv 라이브러리)
from langchain.tools import tool
import arxiv

@tool
def get_source(query: str) -> str:
    """
    Arxiv에서 학술 논문을 검색합니다.
    
    이 도구는 다음과 같은 경우에 사용하세요:
    - 특정 주제에 대한 최신 연구 논문을 찾을 때
    - 저자원 언어 처리, 벤치마크, NLP 관련 연구를 검색할 때
    - 학술적 근거가 필요한 답변을 작성할 때
    
    Args:
        query (str): 검색할 키워드나 주제 (영어로 입력하는 것이 좋습니다)
        
    Returns:
        str: 상위 5개 논문의 제목, 저자, 요약 정보
    """
    try:
        # arxiv 네이티브 라이브러리 사용 (2025 권장)
        client = arxiv.Client(
            page_size=10,
            delay_seconds=3,  # arXiv API 규정 준수
            num_retries=3
        )
        
        search = arxiv.Search(
            query=query,
            max_results=5,
            sort_by=arxiv.SortCriterion.Relevance
        )
        
        results = []
        for result in client.results(search):
            paper_info = f"""
제목: {result.title}
저자: {', '.join([author.name for author in result.authors])}
게시일: {result.published.strftime('%Y-%m-%d')}
요약: {result.summary[:500]}...
링크: {result.entry_id}
---
"""
            results.append(paper_info)
        
        if results:
            return "\n".join(results)
        else:
            return "검색 결과가 없습니다. 다른 키워드로 시도해보세요."
            
    except Exception as e:
        return f"논문 검색 중 오류 발생: {str(e)}\n쿼리를 다시 확인하거나 네트워크 연결을 확인해주세요."

print("✓ Tools 정의 완료 (arxiv 네이티브 라이브러리 사용)")

✓ Tools 정의 완료 (arxiv 네이티브 라이브러리 사용)


In [5]:
# LLM 초기화 (2025 모범 사례: init_chat_model 사용)
from langchain.chat_models import init_chat_model

# init_chat_model: 모든 모델을 한 줄로 초기화하는 통합 인터페이스
# 장점: 
# - 자동 provider 추론 (gpt-4 → openai, claude → anthropic)
# - 런타임에 모델 교체 가능
# - 통일된 API로 여러 provider 지원

llm = init_chat_model(
    model="gpt-4o",           # 또는 "gpt-4", "claude-3-opus-20240229", "gemini-2.5-pro"
    model_provider="openai",  # provider는 자동 추론되지만 명시적으로 지정 가능
    temperature=0.7,          # 창의성 조절 (0.0=결정적, 1.0=창의적)
    timeout=30,               # API 타임아웃 (초)
    max_tokens=2000           # 최대 응답 토큰 수
)

print("✓ LLM 초기화 완료!")

✓ LLM 초기화 완료!


In [6]:
# Agent 생성 (2025 최신 방식: create_agent 사용)
from langchain.agents import create_agent

# create_agent: 프로덕션 레벨의 agent 구현
agent = create_agent(
    model=llm,                    # init_chat_model로 초기화된 모델
    tools=[get_source],           # 사용할 도구 리스트
    system_prompt=SYSTEM_PROMPT   # System prompt (state_modifier가 아님!)
)

print("✓ Agent 생성 완료!")
print("\n사용 가능한 도구:", [tool.name for tool in [get_source]])

✓ Agent 생성 완료!

사용 가능한 도구: ['get_source']


In [7]:
# Agent 실행 예제 쿼리
# KLSBench 연구와 관련된 샘플 질문들

# 예제 1: 저자원 언어 벤치마크 연구
query_1 = "저자원 언어 처리를 위한 벤치마크 구축에 대한 최신 연구를 찾아줘. 특히 low-resource language benchmark 관련 논문이 필요해."

# 예제 2: 한문(Classical Chinese) 처리 연구
query_2 = "Classical Chinese NLP와 한문 자동 처리에 관한 연구 논문을 검색해줘."

# 예제 3: 벤치마크 평가 방법론
query_3 = "NLP 벤치마크의 평가 방법론과 데이터셋 구축 방법에 대한 논문을 찾아줘."

# Agent 실행 (원하는 쿼리 주석 해제하여 실행)
user_query = query_1  # 여기서 query_1, query_2, query_3 중 선택

print(f"질문: {user_query}\n")
print("=" * 80)

# Agent 스트리밍 실행
for chunk in agent.stream(
    {"messages": [("user", user_query)]},
    stream_mode="values"
):
    # 마지막 메시지만 출력
    if "messages" in chunk and chunk["messages"]:
        last_message = chunk["messages"][-1]
        if hasattr(last_message, "content"):
            print(last_message.content)

질문: 저자원 언어 처리를 위한 벤치마크 구축에 대한 최신 연구를 찾아줘. 특히 low-resource language benchmark 관련 논문이 필요해.

저자원 언어 처리를 위한 벤치마크 구축에 대한 최신 연구를 찾아줘. 특히 low-resource language benchmark 관련 논문이 필요해.


제목: Pre-training on high-resource speech recognition improves low-resource speech-to-text translation
저자: Sameer Bansal, Herman Kamper, Karen Livescu, Adam Lopez, Sharon Goldwater
게시일: 2018-09-05
요약: We present a simple approach to improve direct speech-to-text translation
(ST) when the source language is low-resource: we pre-train the model on a
high-resource automatic speech recognition (ASR) task, and then fine-tune its
parameters for ST. We demonstrate that our approach is effective by
pre-training on 300 hours of English ASR data to improve Spanish-English ST
from 10.8 to 20.2 BLEU when only 20 hours of Spanish-English ST training data
are available. Through an ablation study, we find t...
링크: http://arxiv.org/abs/1809.01431v2
---


제목: Indian Legal NLP Benchmarks : A Survey
저자: Prathamesh Kalamkar, Janani 